In [6]:
import os
import sys
import csv
import math
import numpy as np
import pandas as pd
from datetime import  timedelta,datetime
import matplotlib.pyplot as plt
R = 6.371*10**6

def great_circle_dist(lat1,lon1,lat2,lon2):
  lat1 = lat1/180*math.pi
  lon1 = lon1/180*math.pi
  lat2 = lat2/180*math.pi
  lon2 = lon2/180*math.pi
  temp = np.cos(lat1)*np.cos(lat2)*np.cos(lon1-lon2)+np.sin(lat1)*np.sin(lat2)
  if isinstance(temp,np.ndarray):
    temp[temp>1]=1
    temp[temp<-1]=-1
  else:
    if temp>1:
      temp=1
    if temp<-1:
      temp=-1
  theta = np.arccos(temp)
  d = theta*R
  return d

def pairwise_great_circle_dist(latlon_array):
  dist = []
  k = np.shape(latlon_array)[0]
  for i in range(k-1):
    for j in np.arange(i+1,k):
      dist.append(great_circle_dist(latlon_array[i,0],latlon_array[i,1],latlon_array[j,0],latlon_array[j,1]))
  return dist

In [2]:
traj = np.load("FullMobMat.npy")
traj

array([[ 1.00000000e+00,  4.23671283e+01, -7.11138670e+01, ...,
        -7.11138699e+01,  1.55111011e+09,  1.00000000e+00],
       [ 2.00000000e+00,  4.23671198e+01, -7.11138699e+01, ...,
        -7.11138699e+01,  1.55111016e+09,  1.00000000e+00],
       [ 1.00000000e+00,  4.23671198e+01, -7.11138699e+01, ...,
        -7.11141464e+01,  1.55111021e+09,  0.00000000e+00],
       ...,
       [ 2.00000000e+00,  4.23671446e+01, -7.11138669e+01, ...,
        -7.11138669e+01,  1.55206614e+09,  1.00000000e+00],
       [ 1.00000000e+00,  4.23671446e+01, -7.11138669e+01, ...,
        -7.11139228e+01,  1.55206615e+09,  1.00000000e+00],
       [ 2.00000000e+00,  4.23671337e+01, -7.11139228e+01, ...,
        -7.11139228e+01,  1.55206621e+09,  1.00000000e+00]])

In [15]:
def num_sig_places(data,dist):
  loc_x = []; loc_y = []; num_xy=[]; t_xy = []
  for i in range(data.shape[0]):
    if len(loc_x)==0:
      loc_x.append(data[i,1])
      loc_y.append(data[i,2])
      num_xy.append(1)
      t_xy.append(data[i,6]-data[i,3])
    else:
      d = []
      for j in range(len(loc_x)):
        d.append(great_circle_dist(data[i,1],data[i,2],loc_x[j],loc_y[j]))
      index = d.index(min(d))
      if min(d)>dist:
        loc_x.append(data[i,1])
        loc_y.append(data[i,2])
        num_xy.append(1)
        t_xy.append(data[i,6]-data[i,3])
      else:
        loc_x[index] = (loc_x[index]*num_xy[index]+data[i,1])/(num_xy[index]+1)
        loc_y[index] = (loc_y[index]*num_xy[index]+data[i,2])/(num_xy[index]+1)
        num_xy[index] = num_xy[index] + 1
        t_xy[index] = t_xy[index]+data[i,6]-data[i,3]
  return loc_x,loc_y,num_xy,t_xy

def GetStats(traj):
  sys.stdout.write("Calculating the summary stats..." + '\n')
  ObsTraj = traj[traj[:,7]==1,:]
  hours = [datetime.fromtimestamp((ObsTraj[i,3]+ObsTraj[i,6])/2).hour for i in range(ObsTraj.shape[0])]
  hours = np.array(hours)
  home_pauses = ObsTraj[((hours>=19)+(hours<=9))*ObsTraj[:,0]==2,:]
  loc_x,loc_y,num_xy,t_xy = num_sig_places(home_pauses,20)
  home_index = num_xy.index(max(num_xy))
  home_x, home_y = loc_x[home_index],loc_y[home_index]

  ## find starting and ending time
  start_time = datetime.fromtimestamp(traj[0,3])
  start_time = start_time + timedelta(hours=1) - timedelta(minutes=start_time.minute,
                      seconds=start_time.second, microseconds=start_time.microsecond)
  start_stamp = datetime.timestamp(start_time)
  end_time = datetime.fromtimestamp(traj[-1,3])
  end_time = end_time - timedelta(minutes=end_time.minute,
                      seconds=end_time.second, microseconds=end_time.microsecond)
  end_stamp = datetime.timestamp(end_time)
  ## start_time, end_time are exact points (if it is 2019-3-8 11hr, then 11 shouldn't be included)

  ## hourly summary statistics from here
  summary_stats = []
  h = (end_stamp - start_stamp)/60/60
  for i in range(int(h)):
    t0 = start_stamp + i*3600
    t1 = start_stamp + (i+1)*3600
    ## take a subset, the starting point of the last traj <t1 and the ending point of the first traj >t0
    index = (traj[:,3]<t1)*(traj[:,6]>t0)
    temp = traj[index,:]
    p0 = (temp[0,6]-t0)/(temp[0,6]-temp[0,3])
    p1 = (t1-temp[-1,3])/(temp[-1,6]-temp[-1,3])
    temp[0,1] = (1-p0)*temp[0,1]+p0*temp[0,4]
    temp[0,2] = (1-p0)*temp[0,2]+p0*temp[0,5]
    temp[0,3] = t0
    temp[-1,4] = (1-p1)*temp[-1,1] + p1*temp[-1,4]
    temp[-1,5] = (1-p1)*temp[-1,2] + p1*temp[-1,5]
    temp[-1,6] = t1
    current_t = datetime.fromtimestamp(t0)
    year,month,day,hour = current_t.year, current_t.month,current_t.day,current_t.hour
    missing_length = sum((temp[:,6]-temp[:,3])[temp[:,7]==0])/60
    d_home_1 = great_circle_dist(home_x,home_y,temp[:,1],temp[:,2])
    d_home_2 = great_circle_dist(home_x,home_y,temp[:,4],temp[:,5])
    d_home = (d_home_1+d_home_2)/2
    max_dist_home = max(np.concatenate((d_home_1,d_home_2)))
    time_at_home = sum((temp[:,6]-temp[:,3])[d_home<=50])/60
    total_pause_time =  sum((temp[:,6]-temp[:,3])[temp[:,0]==2])/60
    total_flight_time =  sum((temp[:,6]-temp[:,3])[temp[:,0]==1])/60
    dist_traveled = sum(great_circle_dist(temp[:,4],temp[:,5],temp[:,1],temp[:,2]))
    loc_x,loc_y,num_xy,t_xy = num_sig_places(temp[temp[:,0]==2,:],50)
    num_sig = sum(np.array(t_xy)/60>10)
    if temp.shape[0]==1:
      diameter = 0
    else:
      D = pairwise_great_circle_dist(temp[:,[1,2]])
      diameter = max(D)
    summary_stats.append([year,month,day,hour,missing_length,total_pause_time,total_flight_time,time_at_home,max_dist_home,
                         dist_traveled,diameter,num_sig])
  summary_stats = pd.DataFrame(np.array(summary_stats))
  summary_stats.columns = ["year","month","day","hour","missing_time","pause_time","flight_time","home_time","max_dist_home",
                          "dist_traveled","diameter","num_sig_places"]
  return(summary_stats)

In [16]:
out = GetStats(traj)
out

Calculating the summary stats...


,year,month,day,hour,missing_time,pause_time,flight_time,home_time,max_dist_home,dist_traveled,diameter,num_sig_places
0,2019.0,2.0,25.0,11.0,56.833333,53.500000,6.500000,59.000000,75.404747,242.113791,96.387430,1.0
1,2019.0,2.0,25.0,12.0,60.000000,60.000000,0.000000,60.000000,12.003967,0.000000,0.000000,1.0
2,2019.0,2.0,25.0,13.0,60.000000,60.000000,0.000000,60.000000,12.003967,0.000000,0.000000,1.0
3,2019.0,2.0,25.0,14.0,60.000000,60.000000,0.000000,60.000000,12.003967,0.000000,0.000000,1.0
4,2019.0,2.0,25.0,15.0,60.000000,60.000000,0.000000,60.000000,12.003967,0.000000,0.000000,1.0
5,2019.0,2.0,25.0,16.0,60.000000,60.000000,0.000000,60.000000,12.003967,0.000000,0.000000,1.0
6,2019.0,2.0,25.0,17.0,60.000000,60.000000,0.000000,60.000000,12.003967,0.000000,0.000000,1.0
7,2019.0,2.0,25.0,18.0,60.000000,58.333333,1.666667,22.737600,64.056497,52.801727,52.706792,2.0
8,2019.0,2.0,25.0,19.0,60.000000,56.666667,3.333333,0.000000,102.209125,138.770419,138.675483,1.0
9,2019.0,2.0,25.0,20.0,60.000000,60.000000,0.000000,0.000000,102.209125,0.000000,0.000000,1.0
